In [35]:
import pandas as pd
import numpy as np

In [36]:
weather_2023_df = pd.read_csv("../data/weather_2023.csv")
weather_2023_df['DATE'] = pd.to_datetime(weather_2023_df['DATE'], format="%Y-%m-%dT%H:%M:%S")



/var/folders/bc/1syg3kl11695l0f9yqy1xfcr0000gn/T/ipykernel_10627/3551014268.py:1: DtypeWarning: Columns (36,42,43,48,52,88) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_2023_df = pd.read_csv("../data/weather_2023.csv")


In [37]:
weather_2023_df.columns

Index(['STATION', 'DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION',
       'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'WND', 'CIG',
       'VIS', 'TMP', 'DEW', 'SLP', 'AA1', 'AA2', 'AA3', 'AB1', 'AD1', 'AE1',
       'AH1', 'AH2', 'AH3', 'AH4', 'AH5', 'AH6', 'AI1', 'AI2', 'AI3', 'AI4',
       'AI5', 'AI6', 'AJ1', 'AK1', 'AL1', 'AM1', 'AN1', 'AT1', 'AT2', 'AT3',
       'AT4', 'AT5', 'AU1', 'AU2', 'AW1', 'AW2', 'AW3', 'AX1', 'AX2', 'AX3',
       'AX4', 'GA1', 'GA2', 'GA3', 'GD1', 'GD2', 'GD3', 'GE1', 'GF1', 'KA1',
       'KA2', 'KB1', 'KB2', 'KB3', 'KC1', 'KC2', 'KD1', 'KD2', 'KE1', 'KG1',
       'KG2', 'MA1', 'MD1', 'MF1', 'MG1', 'MH1', 'MK1', 'MW1', 'OC1', 'OD1',
       'OE1', 'OE2', 'OE3', 'RH1', 'RH2', 'RH3', 'WA1', 'REM', 'EQD'],
      dtype='object')

In [38]:
hourly_weather = weather_2023_df.loc[weather_2023_df['REPORT_TYPE'] == 'FM-15', :]
hourly_weather.shape
hourly_weather['DATE']

5       2023-01-01 00:51:00
8       2023-01-01 01:51:00
10      2023-01-01 02:51:00
13      2023-01-01 03:51:00
16      2023-01-01 04:51:00
                ...        
11837   2023-12-31 19:51:00
11838   2023-12-31 20:51:00
11839   2023-12-31 21:51:00
11840   2023-12-31 22:51:00
11841   2023-12-31 23:51:00
Name: DATE, Length: 8757, dtype: datetime64[ns]

In [19]:
hourly_weather['DATE'].dt.date

5        2023-01-01
8        2023-01-01
10       2023-01-01
13       2023-01-01
16       2023-01-01
            ...    
11837    2023-12-31
11838    2023-12-31
11839    2023-12-31
11840    2023-12-31
11841    2023-12-31
Name: DATE, Length: 8757, dtype: object

In [20]:
columns = ['date','hour', 'wind_speed', 'dew_point', 'atmospheric_pressure', 'temperature']
hourly_weather_2023_filtered = pd.DataFrame(columns=columns)


In [21]:

hourly_weather_2023_filtered['wind_speed'] = hourly_weather['WND'].apply(lambda x: int((x.split(',')[-2]))/10).replace(999.9, np.nan)
hourly_weather_2023_filtered['temperature'] = hourly_weather['TMP'].apply(lambda x: int((x.split(',')[0]))/10).replace(999.9, np.nan)
hourly_weather_2023_filtered['dew_point'] = hourly_weather['DEW'].apply(lambda x: int((x.split(',')[0]))/10).replace(999.9, np.nan)  #precipitation
hourly_weather_2023_filtered['atmospheric_pressure'] = hourly_weather['SLP'].apply(lambda x: int((x.split(',')[0]))/10).replace(9999.9, np.nan)  #precipitation
hourly_weather_2023_filtered['date'] = [i for i in hourly_weather['DATE'].dt.date]
hourly_weather_2023_filtered['hour'] = hourly_weather['DATE'].dt.hour.apply(lambda x: f'{x:02}')




In [23]:
hourly_weather_2023_filtered.head(30)

,date,hour,wind_speed,dew_point,atmospheric_pressure,temperature
5,2023-01-01,00,0.0,9.4,1009.8,10.0
8,2023-01-01,01,3.1,11.7,1008.6,12.8
10,2023-01-01,02,0.0,11.7,1008.6,12.2
13,2023-01-01,03,2.6,11.1,1007.8,12.8
16,2023-01-01,04,0.0,11.1,1007.0,12.2
22,2023-01-01,05,2.6,10.6,1007.2,11.7
23,2023-01-01,06,3.6,10.0,1006.9,11.7
24,2023-01-01,07,2.6,10.0,1007.8,11.1
25,2023-01-01,08,4.1,9.4,1008.1,11.7
26,2023-01-01,09,2.1,8.9,1008.5,11.1


In [25]:
hourly_weather_2023_filtered = hourly_weather_2023_filtered.ffill()

In [31]:
hourly_weather_2023_filtered.to_csv("../data/hourly_weather_2023.csv", index=False)


In [32]:
hourly_weather['DATE']

5       2023-01-01 00:51:00
8       2023-01-01 01:51:00
10      2023-01-01 02:51:00
13      2023-01-01 03:51:00
16      2023-01-01 04:51:00
                ...        
11837   2023-12-31 19:51:00
11838   2023-12-31 20:51:00
11839   2023-12-31 21:51:00
11840   2023-12-31 22:51:00
11841   2023-12-31 23:51:00
Name: DATE, Length: 8757, dtype: datetime64[ns]

In [33]:
weather_2023_df['DATE'] = pd.to_datetime(weather_2023_df['DATE'])

# Filter for rows where 'DATE' is 2023-08-14
filtered_weather_2023_df = weather_2023_df[weather_2023_df['DATE'].dt.date == pd.to_datetime('2023-08-14').date()]

In [34]:
filtered_weather_2023_df

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OD1,OE1,OE2,OE3,RH1,RH2,RH3,WA1,REM,EQD
7175,72505394728,2023-08-14 00:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09508/13/23 19:51:01 METAR KNYC 140051Z VRB...,NaN
7176,72505394728,2023-08-14 02:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09808/13/23 21:51:02 METAR KNYC 140251Z 260...,NaN
7177,72505394728,2023-08-14 03:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET09208/13/23 22:51:02 METAR KNYC 140351Z VRB...,NaN
7178,72505394728,2023-08-14 04:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10208/13/23 23:51:02 METAR KNYC 140451Z 280...,NaN
7179,72505394728,2023-08-14 04:59:00,6,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",SOD,KNYC,V030,...,NaN,"1,24,01341,210,9999,5","2,24,00447,270,9999,5","3,24,00210,999,9999,5","024,X,093,D,4","024,N,043,D,4","024,M,067,D,4",NaN,SOD76024 HR PRECIPITATION (IN): 0.69 SNOW DEPT...,NaN
7180,72505394728,2023-08-14 05:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11908/14/23 00:51:02 METAR KNYC 140551Z 280...,NaN
7181,72505394728,2023-08-14 06:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10108/14/23 01:51:02 METAR KNYC 140651Z 000...,NaN
7182,72505394728,2023-08-14 07:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10108/14/23 02:51:02 METAR KNYC 140751Z 000...,NaN
7183,72505394728,2023-08-14 08:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10708/14/23 03:51:02 METAR KNYC 140851Z 000...,NaN
7184,72505394728,2023-08-14 09:51:00,7,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,KNYC,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10108/14/23 04:51:02 METAR KNYC 140951Z 000...,NaN
